# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube dans une collection MongoDB.

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
Toutes les questions sont à réalisés via des requêtes vers Mongo :
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [3]:
!pip install pymongo
import pandas as pd
import pymongo

In [147]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['youtube']

In [5]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [132]:
df_youtube_cat = pd.read_json("./data/US_category_id_v2.json", orient="values")

for new_col in ["channelId", "title", "assignable"]:
    df_youtube_cat.loc[new_col] = df_youtube_cat.loc["snippet"].apply(lambda x: x[new_col])

df_youtube_cat = df_youtube_cat.T.drop(columns=["snippet"])
df_youtube_cat.rename(columns={"id":"category_id"}, inplace=True)
df_youtube_cat.category_id = df_youtube_cat.category_id.apply(pd.to_numeric)

print(df_youtube_cat.info())
df_youtube_cat.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 6 columns):
kind           32 non-null object
etag           32 non-null object
category_id    32 non-null int64
channelId      32 non-null object
title          32 non-null object
assignable     32 non-null object
dtypes: int64(1), object(5)
memory usage: 1.8+ KB
None


,kind,etag,category_id,channelId,title,assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


### Importer les données

In [135]:
df = pd.merge(df_youtube, df_youtube_cat, on=["category_id"])

In [136]:
df.head()

,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,channelId,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True


## Question 1  

In [141]:
df[df["channel_title"] == "Apple"]

,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,channelId,title_y,assignable
1601,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1617,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,12200526,258842,44339,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,14.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1636,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,15731493,321403,57528,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,15.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1657,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,18082737,359392,64933,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,16.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1675,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,19707391,381919,69465,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,17.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1689,A9k88sMyiJM,Apple Watch Series 3 + Apple Music — Roll — Apple,Apple,28,Apple|airpods|apple watch|apple watch series 3...,782543,14647,1489,0,https://i.ytimg.com/vi/A9k88sMyiJM/default.jpg,18.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1709,A9k88sMyiJM,Apple Watch Series 3 + Apple Music — Roll — Apple,Apple,28,Apple|airpods|apple watch|apple watch series 3...,3987580,26021,8719,0,https://i.ytimg.com/vi/A9k88sMyiJM/default.jpg,19.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1730,A9k88sMyiJM,Apple Watch Series 3 + Apple Music — Roll — Apple,Apple,28,Apple|airpods|apple watch|apple watch series 3...,4112187,26609,8982,0,https://i.ytimg.com/vi/A9k88sMyiJM/default.jpg,20.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1871,REZl-ANYKKY,iPhone 8 Plus — Portraits of Her — Apple,Apple,28,Apple|new|iPhone|iPhone 8|iPhone 8 Plus|iOS 11...,117564,1240,459,0,https://i.ytimg.com/vi/REZl-ANYKKY/default.jpg,3.10,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
1884,REZl-ANYKKY,iPhone 8 Plus — Portraits of Her — Apple,Apple,28,Apple|new|iPhone|iPhone 8|iPhone 8 Plus|iOS 11...,163024,1538,539,0,https://i.ytimg.com/vi/REZl-ANYKKY/default.jpg,4.10,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True


## Question 2

In [142]:
df["category_id"].unique()

array([24, 28, 22, 23,  1, 26, 10, 25, 17, 27,  2, 20, 15, 29, 19, 43],
      dtype=int64)

## Question 3

## Question 4

## Question 5

## Question 6 

In [153]:
import numpy as np

def createDocsFromDF(df, collection = None, insertToDB=False):
    docs = [] 
    fields = [col for col in df.columns]
    for i in range(len(df)):
        doc = {col:df[col][i] for col in df.columns if col != 'index'}
        for key, val in doc.items():
            # we have to do this, because mongo does not recognize these np. types
            if type(val) == np.int64:
                doc[key] = int(val)
            if type(val) == np.float64:
                doc[key] = float(val)
            if type(val) == np.bool_:
                doc[key] = bool(val)
        docs.append(doc) 
    if insertToDB and collection:
        database.collection.insert_many(docs)
    return docs

createDocsFromDF(df, collection="us-videos", insertToDB=True)

[{'video_id': 'XpVt6Z1Gjjo',
  'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Entertainment',
  'assignable': True,
  '_id': ObjectId('605bada302ca5054d92dd975')},
 {'video_id': '4MkC65emkG4',
  'titl